# Interpolation and Extrapolation

This lecture follows closely [Numerical Recipes](https://numerical.recipes/) 2nd Edition in C and 3rd Edition in C++, Chapter 3 "Interpolation and Extrapolation".

## Introduction

In scientific computing and machine learning, interpolation and extrapolation are fundamental tools for estimating function values at new data points based on known information.
* In machine learning, all standard supervised learning tasks can be viewed as interpolation problems in high-dimensional space. Here, models predict outputs **within the range** of their training data.
* However, when attempting to make predictions outside this range, we face significant challenges in making reliable extrapolations.
  Extrapolation is a particularly challenging task because models typically lack information  beyond their training data.

### Interpolation Methods

Interpolation plays a crucial role in scientific computing and machine learning by estimating function values at new data points based on known information.

* Polynomial interpolation is versatile but can exhibit significant oscillations, particularly at the edges of data (Runge's phenomenon).
* This can be mitigated by using rational functions, which offer more stable estimates and are better suited to handle asymptotic behavior.
* Spline interpolation, especially cubic splines, is valued for its smoothness and continuity up to the second derivative. This makes it effective for applications requiring a smooth fit.

### Challenges with Extrapolation

Extrapolation remains a difficult task, yet physics-informed machine learning (PIML) presents a promising avenue.
By embedding known physical laws, such as ordinary differential equations (ODEs), into models, PIML enables extrapolation that aligns with fundamental constraints. This allows for meaningful extensions of predictions beyond the observed data range.

### Distinguishing Interpolation and Function Approximation

Interpolation and function approximation are related but distinct tasks:

* Interpolation estimates values at specified points within a given dataset.
* In contrast, function approximation creates a simplified function to replace a more complex one.
  This approach can be used to sample additional points as needed.
* (See [Numerical Recipes](https://numerical.recipes/) Chapter 5 for function approximation.)

### Limitations of Interpolation

Even the most sophisticated interpolation schemes can fail when faced with pathological functions.
For instance, consider a function that behaves smoothly except for a slight singularity at a certain point:
\begin{align}
f(x) = 3x^2 + \frac{1}{\pi^4}\ln\left[(\pi - x)^2\right] + 1
\end{align}

Interpolation based on values close to but not precisely at that singularity will likely produce an inaccurate result.

In [ ]:
import numpy as np

def f(x):
    return 3 * x**2 + np.log((np.pi - x)**2) / np.pi**4 + 1

x1 = np.linspace(3.13, 3.16, 3+1)
x2 = np.linspace(3.13, 3.16, 30+1)
x3 = np.linspace(3.13, 3.16, 300+1)
x4 = np.linspace(3.13, 3.16, 3000+1)

In [ ]:
from matplotlib import pyplot as plt

plt.plot(x4, f(x4),       label='3001 points')
plt.plot(x3, f(x3), '--', label='301 points')
plt.plot(x2, f(x2), 'o:', label='31 points')
plt.plot(x1, f(x1), 'o-', label='4 points')
plt.legend()

These cases highlight the importance of having some error estimates in interpolation routines.

## Preliminaries: Searching an Ordered Table

In many interpolation tasks, especially with irregularly sampled data, the process begins with a critical first step: identifying the nearest points surrounding the target interpolation value.

Unlike regularly spaced data on a uniform grid, where adjacent points are easy to locate by simple indexing, randomly sampled or unevenly spaced data requires additional steps to find nearby values.
This searching step can be as computationally intensive as the interpolation itself, so efficient search methods are essential to maintain overall performance.

In Numerical Recipes, two primary methods are presented for this purpose: bisection and hunting.
Each is suited to different scenarios, depending on whether interpolation points tend to be close to one another or scattered randomly.

### Linear Search

As a reference, we will implement a linear search:

In [ ]:
def linear(xs, target):
    for l in range(len(xs)): # purposely use for-loop to avoid C optimization in numpy
        if xs[l] >= target:
            return l-1

In [ ]:
import numpy as np

for _ in range(10):
    xs = np.sort(np.random.uniform(0, 100, 10))
    v  = np.random.uniform(min(xs), max(xs))
    i  = linear(xs, v)
    print(f'{xs[i]} <= {v} < {xs[i+1]}')